## Blur training

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# -------- PATHS --------
BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_ROOT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models/upd"

IMAGE_SIZE = (256, 256)
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

STYLES = ["blur","night_vis","poster", "outline"]

# -------- HELPERS --------
def load_images_from_folder(folder, size=IMAGE_SIZE):
    imgs = []
    files = sorted([
        f for f in os.listdir(folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))
    ])
    for fname in files:
        path = os.path.join(folder, fname)
        img = cv2.imread(path)
        if img is None:
            print("Skipping unreadable:", fname)
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        img = img.astype("float32") / 255.0
        imgs.append(img)
    return np.array(imgs)


def build_unet_autoencoder(input_shape=(256, 256, 3)):
    inp = Input(shape=input_shape)

    # Encoder
    c1 = Conv2D(32, (3,3), activation="relu", padding="same")(inp)
    p1 = MaxPooling2D((2,2))(c1)

    c2 = Conv2D(64, (3,3), activation="relu", padding="same")(p1)
    p2 = MaxPooling2D((2,2))(c2)

    c3 = Conv2D(128, (3,3), activation="relu", padding="same")(p2)
    p3 = MaxPooling2D((2,2))(c3)

    bn = Conv2D(256, (3,3), activation="relu", padding="same")(p3)

    # Decoder
    u3 = UpSampling2D((2,2))(bn)
    u3 = Concatenate()([u3, c3])
    d3 = Conv2D(128, (3,3), activation="relu", padding="same")(u3)

    u2 = UpSampling2D((2,2))(d3)
    u2 = Concatenate()([u2, c2])
    d2 = Conv2D(64, (3,3), activation="relu", padding="same")(u2)

    u1 = UpSampling2D((2,2))(d2)
    u1 = Concatenate()([u1, c1])
    d1 = Conv2D(32, (3,3), activation="relu", padding="same")(u1)

    out = Conv2D(3, (3,3), activation="sigmoid", padding="same")(d1)

    model = Model(inp, out)
    model.compile(optimizer=Adam(1e-4), loss="mae")
    return model


# -------- LOAD INPUT ONCE --------
print("Loading INPUT images...")
input_images = load_images_from_folder(BASE_INPUT)
print("Input shape:", input_images.shape)


# -------- TRAIN PER STYLE --------
for style in STYLES:
    print("\n==============================")
    print(f" TRAINING STYLE: {style.upper()}")
    print("==============================")

    style_folder = os.path.join(STYLE_ROOT, style)
    style_images = load_images_from_folder(style_folder)   # ← FIXED

    print("Style images shape:", style_images.shape)

    if len(style_images) == 0:
        print("No images in", style_folder)
        continue

    if len(style_images) != len(input_images):
        print("COUNT MISMATCH:", len(input_images), "vs", len(style_images))
        continue

    X_train, X_val, y_train, y_val = train_test_split(
        input_images, style_images,
        test_size=0.15,
        random_state=42
    )

    model = build_unet_autoencoder()

    save_path = os.path.join(MODEL_SAVE_DIR, f"autoencoder_{style}_upd.keras")

    callbacks = [
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        ModelCheckpoint(save_path, monitor="val_loss", save_best_only=True)
    ]

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=2,
        callbacks=callbacks,
        verbose=1
    )

    print(f"Saved updated model for '{style}' to: {save_path}")

print("\nAll updated models trained.")

Loading INPUT images...
Input shape: (5, 256, 256, 3)

 TRAINING STYLE: BLUR
Style images shape: (5, 256, 256, 3)
Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 0.3003 - val_loss: 0.2919
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2922 - val_loss: 0.2807
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2870 - val_loss: 0.2679
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2799 - val_loss: 0.2579
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2737 - val_loss: 0.2465
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2666 - val_loss: 0.2346
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2618 - val_loss: 0.2215
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2551 - val_loss: 0.2087
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2472 - val_loss: 0.1981
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2402 - val_loss: 0.1826
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.2322 -

## Outline training

In [1]:
def outline_soft_colored(img, levels=6):
    """
    Soft outline + full natural color.
    No blue tint, no grayscale artifacts.
    """

    # Convert BGR → RGB for consistent color math
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # -------------------------------------------------------
    # 1. Mild Posterization (keeps colors but simplifies them)
    # -------------------------------------------------------
    x = img_rgb.astype(np.float32) / 255.0
    x = np.floor(x * levels) / levels
    poster = (x * 255).astype(np.uint8)

    # -------------------------------------------------------
    # 2. Smooth colors (clean, cartoon-like)
    # -------------------------------------------------------
    smooth = cv2.bilateralFilter(poster, d=7, sigmaColor=50, sigmaSpace=50)

    # -------------------------------------------------------
    # 3. Extract EDGES (but do NOT convert entire image to grayscale!)
    # -------------------------------------------------------
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 60, 140)

    # Soften outlines
    edges_blur = cv2.GaussianBlur(edges, (5, 5), 0)

    # -------------------------------------------------------
    # 4. Convert edges to LIGHT GRAY (neutral, no tint)
    # -------------------------------------------------------
    edges_gray_rgb = cv2.cvtColor(edges_blur, cv2.COLOR_GRAY2RGB)

    # Invert the mask and lighten it
    # This produces thin, light-gray outlines
    outlines = 255 - edges_gray_rgb      # white where no edges
    outlines = outlines * 0.40           # scale intensity
    outlines = outlines.astype(np.uint8)

    # -------------------------------------------------------
    # 5. Blend color + outlines
    # -------------------------------------------------------
    final = cv2.addWeighted(smooth, 1.0, outlines, 0.5, 0)

    # Convert back to BGR for saving with cv2.imwrite
    return cv2.cvtColor(final, cv2.COLOR_RGB2BGR)
